<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_16/1_Gym_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Знакомство с Gym

В этом уроке мы познакомимся с удобной средой для проведения RL экспериментов под названием Gym (https://gym.openai.com/). По аналогии с тем, как для обучения с учителем нам надо где-то брать обучающие датасеты, для обучения с подкреплением нам нужно иметь среду с определёнными правилами (или симуляцию это среды), в которой действует агент. 

в Gym содержатся реализации различных игр и других симуляций, на которых можно проводить эксперименты в области RL. Кроме самих симуляций в Gym имеется удобная обёртка для доступа к симуляции: мы сразу можем оперировтаь в терминах RL (состояние, действие, итд).

### Загрузка библиотек

Загружаем библиотеку gym

In [1]:
import gym

### Создание игровой среды

С помощью функции `gym.make` мы можем создать симулятор необходимой нам игры. Рассмотрим пример игры `Frozen Lake`. Здесь есть ледяное поле и ямы (holes). Цель игры дойти до целевой позиции, не упав в яму. У этой игры есть парамтер `is_slippery`, который означает, "будет ли лёд скользким" -- будет ли среда всегда со стопроцентной вероятностью подчиняться нашему действию. Для простоты отключим этот флаг (куда захотели пойти, там и оказались).

Другие доступные симуляции: https://gym.openai.com/envs/

Посмотрим, сколько есть возможных действий и состояний в нашей игре. Кол-во состояний 16, так как столько ячеек в поле (потенциальных позиций для агента). А действий 4 (4 направления).

In [2]:
env = gym.make('FrozenLake-v0', is_slippery=False)

NUM_STATES = env.observation_space.n
NUM_ACTIONS = env.action_space.n

print('States: {}'.format(NUM_STATES))
print('Actions: {}'.format(NUM_ACTIONS))

States: 16
Actions: 4


### Основные функции Gym

С помощью `env.reset()` можно перезапустить среду в исходное состояние (на начало эпизода). Эта функция также вернет начальное состояние.

В нашем случае состояние это просто число -- индекс соответствующей ячейки, где находится робот.

In [3]:
 s = env.reset()
 print(s)

0


С помощью функции `env.render()` можно визуализировать текущее состояние среды. В Colab это не всегда можно сделать довольно просто (в случае сложных симуляций), но в случае Frozen Lake это просто напечатанный текст с нашим полем 4x4. S - start, F - frozen, H - hole, G - goal. Маркером указано положение робота.

In [4]:
env.render()


SFFF
FHFH
FFFH
HFFG


Действия тоже кодируруются соответствующим индексом.
Можно, например, выбрать случайное действие с попомщью функции `env.action_space.sample()`

In [5]:
a = env.action_space.sample()
print(a)

1


Чтобы совершить действие `a` нужно вызвать функцию `env.step(a)`. Эта функция вернет новое состояние (`s1`), в которое мы перешли, награду `r`, информацию о том, завершилась ли игра (`done`) и другую менее важную информацию.

In [9]:
s1, r, done, _ = env.step(a)

print('New state: ', s1)
print('Reward: ', r)
print('Done? ', done)

New state:  5
Reward:  0
Done?  True


Посмотрим, как теперь выглядит текущее состояние.

In [10]:
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


### Запуск симуляции

Теперь у нас есть все знания, чтобы сыграть целый игровой эпизод с помощью Gym. Сначала встаём в стартовую позицию `env.reset()`. Потом в цикле совершаем шаги и рисуем промежуточные состояние с помощью `env.render()`. На каждом шаге нам как-то надо выбрать действие. Выбор действия обернём в функцию `a = policy(s)`. В идеале мы должны руководстсоваться некой стратегией, но в этом простом примере будем выбирать случайные действия `a`. После выбора действия `a` делаем шаг `env.step(a)` (сообщаем среде наше желание сделать действие). Если после определённого шага среда вернула `done=True`, значит произошел конец эпизода (упали в яму или дошли до цели). Если мы дошли до цели, то на последнем шаге мы должны были получить ненулевую награду.

In [136]:
s = env.reset()
def policy():
    
    # non случайная стратегия
    
    happy_path = [1,1,2,1,2,2]
    for i in happy_path:
      env.render()
      
      s, r, done, _ = env.step(i) 
      print('Reward = {}'.format(r))
      if done:
        env.render()
        print('Final reward = {}'.format(r))
        break
        
policy()
env.close()


SFFF
FHFH
FFFH
HFFG
Reward = 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward = 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward = 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
Reward = 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward = 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
Reward = 1.0
  (Right)
SFFF
FHFH
FFFH
HFFG
Final reward = 1.0


**[Задание 1]** Запрограммируйте бота вручную на совершение таких действий, при которых он дойдёт до цели и получит ненулевую награду. Для этого измените лишь функцию `policy()`. Просто создайте набор правил (стратегию) -- какое действие надо совершить в зависимости от состояния. Проведите симуляцию с использованием вашей стратегии и посмотрите на результат.

In [106]:
#env.render()
actions = {
    'Left': 0,
    'Down': 1,
    'Right': 2, 
    'Up': 3
}
 
print('---- winning sequence ------ ')
winning_sequence = (2 * ['Right']) + (3 * ['Down']) + ['Right']
print(winning_sequence)
 
env = gym.make('FrozenLake-v0', is_slippery=False)
env.reset()
env.render()
 
for a in winning_sequence:
    new_state, reward, done, info = env.step(actions[a])
    print()
    env.render()
    print("Reward: {:.2f}".format(reward))
    print(info)
    if done:
        break  
 
print()

---- winning sequence ------ 
['Right', 'Right', 'Down', 'Down', 'Down', 'Right']

SFFF
FHFH
FFFH
HFFG

  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 0.00
{'prob': 1.0}

  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 0.00
{'prob': 1.0}

  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.00
{'prob': 1.0}

  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.00
{'prob': 1.0}

  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.00
{'prob': 1.0}

  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 1.00
{'prob': 1.0}



In [107]:
env.reset()

0

In [108]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [121]:
env.step(2)


(15, 1.0, True, {'prob': 1.0})

In [122]:
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


In [47]:
actions['Left']

0